In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms,datasets


In [2]:
BATCH_SIZE = 6400
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print (DEVICE)
EPOCHS = 10

cuda


In [3]:
pipline =transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307),(0.3081)),
])

In [4]:

train_set = datasets.MNIST("data",train=True,download=True,transform=pipline)
test_set = datasets.MNIST("data",train=False,download=True,transform=pipline)


In [5]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_set,batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE, shuffle=True)

In [6]:
class Digit(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,10,3)
        self.conv2 = nn.Conv2d(10,20,3)
        self.fc1 = nn.Linear(20*10*10, 500)
        self.fc2 = nn.Linear(500, 10)
    def forward(self, x):
        inpute_size = x.size(0)
        x = self.conv1(x)  
        x = F.relu(x) #非线性
        x = F.max_pool2d(x,2,2)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(inpute_size,-1) # -1自动计算
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x,dim=1) #计算分类后的概率
        return output


In [7]:
model = Digit().to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [8]:
def train_modle(model ,device , train_loader , optimizer , epoch):
    model.train()
    for batch_index, (data ,target) in enumerate(train_loader):
        data, target = data.to(device),target.to(device)
        optimizer.zero_grad()
        output = model(data)
        lose = F.cross_entropy(output, target)
       # pred = output.max(1,keepdim= True)
        lose.backward()
        optimizer.step()
        if batch_index % 3000 ==  0 :
            print ("train :{} \t loss : {:.6f}".format(epoch ,lose.item()))



In [9]:
def test_modle(model , device , test_loader):
    model.eval()
    correct = 0.0  
    test_loss =0.0
    with torch.no_grad() :
        for data , target in test_loader :
            data , target =data.to(device), target.to(device)
            output = model(data)
            test_loss +=F.cross_entropy(output , target).item()
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)
        print ("loss: {:.4f},rate : {:.3f}\n".format(test_loss,100*correct/len(test_loader.dataset)))

In [10]:
for epoch in range (1,EPOCHS+1):
    train_modle(model ,DEVICE ,train_loader ,optimizer,epoch)
    test_modle(model , DEVICE,test_loader)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (6400x24200 and 2000x500)